In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import itertools
import json
import os

import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import sklearn
import sklearn.cluster
import torch
import tqdm
import umap
# import umap.parametric_umap

import celltrip
  

mpl.rcParams['pdf.fonttype'] = 42


2026-01-09 19:49:04.474567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load Data

In [3]:
# Read data
adata, = celltrip.utility.processing.read_adatas('s3://nkalafut-celltrip/ExpVal/expression.h5ad', backed=True)
sample_mask = np.random.rand(adata.shape[0]) < .1  # 76,855 total, REMOVE
adata = adata[sample_mask]

# Correct variables
adata.obs['age_orig'] = adata.obs['age']
adata.obs['age'] = adata.obs['age'].apply(lambda x: {  # Translate ages
    i+1: v for i, v in enumerate((85, 95, 100, 105, 110, 125, 145, 155))}[x])
adata.obs['subtype_orig'] = adata.obs['subtype']
adata.obs['subtype'] = adata.obs['subtype'].map(lambda s: ' '.join(s.split(' ')[:2]), na_action='ignore')  # Shorten subtypes

# Normalize data for visualization
sim_data = adata.X[:]
sim_data_norm = 10_000 * sim_data / sim_data.sum(axis=1)

# Load manager
prefix, training_step = 's3://nkalafut-celltrip/checkpoints/ExpVal-251120', 800
manager = celltrip.manager.BasicManager(
    policy_fname=f'{prefix}-{training_step:04}.weights',
    preprocessing_fname=f'{prefix}.pre',
    mask_fname=f'{prefix}.mask',
    device='cuda')
manager.set_modalities([adata], suppress_warning=True)


/tmp/ipykernel_2950/2666922103.py:7: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['age_orig'] = adata.obs['age']


In [4]:
adata.obs.columns

Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'age',
       'percent_mito', 'percent_ribo', 'cellID', 'doubletFinderOutput',
       'seurat_clusters', 'HarmonyClusters2.0', 'HarmonyClusters2.5',
       'CCAClusters2.0', 'CCAClusters2.5', 'CCA_2.0_class', 'CCA_2.5_class',
       'Harmony_2.0_class', 'Harmony_2.5_class', 'MNNClusters2.0',
       'MNNClusters2.5', 'MNN_2.0_class', 'MNN_2.5_class',
       'indForSubClustering', 'Harmony_SubClusters', 'Harmony_subclass',
       'Harmony_subclass_v2', 'Harmony_2.5_class_v2', 'subtype', 'subtype_v2',
       'Train', 'age_orig', 'subtype_orig'],
      dtype='object')

# Simulate

In [5]:
# Simulate to steady state
manager.reset_env()
manager.simulate(skip_time=512., impute=False)
manager.save_state('steady')
steady_gex, = manager.get_state()

100%|██████████| 5120/5120 [01:53<00:00, 45.18it/s]


## Visualize

In [6]:
# # Reduce observed
# # red = umap.parametric_umap.UMAP(
# red = umap.UMAP(
#     # low_memory=True,
#     # random_state=42,
# )
# sim_data_embed = red.fit_transform(sim_data_norm)
# sim_data_embed = pd.DataFrame(sim_data_embed, index=adata.obs_names, columns=[f'UMAP-{i+1}' for i in range(sim_data_embed.shape[1])])
# sim_data_embed['subtype'] = adata.obs['subtype']
# sim_data_embed['sample'] = adata.obs['sample']
# sim_data_embed['age'] = adata.obs['age']
# sim_data_embed['source'] = 'Observed'

# # Reduce CellTRIP
# steady_gex_embed = red.transform(steady_gex)  # np.concat([sim_data, steady_gex], axis=0)
# steady_gex_embed = pd.DataFrame(steady_gex_embed, index=adata.obs_names, columns=[f'UMAP-{i+1}' for i in range(steady_gex_embed.shape[1])])
# steady_gex_embed['subtype'] = adata.obs['subtype']
# steady_gex_embed['sample'] = adata.obs['sample']
# steady_gex_embed['age'] = adata.obs['age']
# steady_gex_embed['source'] = 'CellTRIP'

In [7]:
# # Concatenate
# concat_embed = pd.concat([sim_data_embed, steady_gex_embed], axis=0)

# # Plot observed vs. CellTRIP
# fig, ax = plt.subplots(figsize=(6, 6))
# var = 'age'  # 'subtype'

# sns.scatterplot(concat_embed, x='UMAP-1', y='UMAP-2', hue=var, style='source', s=5, legend=True, ax=ax)
# sns.despine(ax=ax)
# # Annotate
# # var_means = concat_embed.groupby(var, observed=True)[['UMAP-0', 'UMAP-1']].median()
# # for var, r in var_means.iterrows():
# #     ax.text(*r.to_numpy(), var, ha='center', va='center', fontsize='xx-small', transform=ax.transData)

# # Save
# fig.savefig('../plots/expval/age_umap.pdf')

# Knockdown

In [8]:
# Parameters
samples_to_perturb = adata.obs['sample'].unique()

# Manual assignment
# genes_to_perturb = (
#     'RAPGEF4', 'CHD8',  # Experimental genes for neuronal maturation
#     'MEIS2', 'NFIA', 'NFIB',  # Known TFs for neuronal maturation
# )

# Permutations
n_samples = 1_000
n_samples_per_gene = 3
# Generate permutations
np.random.seed(42)
n_genes = adata.shape[1]
permutation_matrix = np.zeros((n_samples, n_genes), dtype=bool)
idx = np.random.rand(*permutation_matrix.shape).argpartition(-n_samples_per_gene, axis=0)[-n_samples_per_gene:]  # Select `n_samples_per_gene` for each gene
permutation_matrix[idx, np.indices(idx.shape)[1]] = True
# CLI
est_time = len(samples_to_perturb) * (25 + 4 * n_samples) / 60**2  # timepoints * (25 sec/init + 4 sec/pert * n_samples)
print(f'Perturbation will take ~{est_time:.2f}h with ~{n_genes*n_samples_per_gene/n_samples:.0f} genes per permutation')
# Final gene lists
genes_to_perturb = [tuple(adata.var_names[r].tolist()) for r in permutation_matrix]


Perturbation will take ~11.18h with ~47 genes per permutation


In [9]:
# # Initialize
# pert_results = []
# for sample in samples_to_perturb:
#     # Filter adata
#     adata_sample = adata[adata.obs['sample']==sample]
#     sample_age = adata_sample.obs['age'].iloc[0]
#     manager.set_modalities([adata_sample], suppress_warning=True)
#     identifier = f'{sample_age} ({sample})'

#     # Simulate to steady state
#     print(f'Simulating sample {sample} of age {sample_age}...')
#     manager.reset_env()
#     manager.simulate(skip_time=512., impute=False)
#     manager.save_state('steady')
#     steady_gex, = manager.get_state()

#     # Get control
#     time, states = manager.simulate_perturbation(skip_time=512., time=128.)
#     control_gex, = manager.get_state()

#     # Revert to previous state
#     manager.load_state('steady')
#     manager.clear_perturbations()

#     # Perturb chosen genes
#     for gene in tqdm.tqdm(genes_to_perturb):
#         # Simulate perturbation
#         # print(f'Perturbing gene(s) {gene}...')
#         gene = (gene,) if isinstance(gene, str) else gene  # Always list-like
#         manager.add_perturbation(gene, modality=0, feature_targets=0)
#         time, states = manager.simulate_perturbation(skip_time=512., time=128., progress_bar=False, impute=False)
#         knocked_gex, = manager.get_state()

#         # Get statistics
#         diff = pd.DataFrame(index=adata_sample.obs_names)
#         diff['subtype'] = adata_sample.obs['subtype']
#         diff['Steady GES'] = np.sqrt(np.square(knocked_gex - steady_gex).sum(axis=1))
#         diff['Control GES'] = np.sqrt(np.square(knocked_gex - control_gex).sum(axis=1))
#         diff_means = diff.groupby('subtype', observed=True).mean()
#         for subtype in ['All'] + adata_sample.obs['subtype'].unique().tolist():
#             if subtype == 'All': diff_results = diff.drop(columns=['subtype']).mean(axis=0).to_dict()
#             else: diff_results = diff_means.loc[subtype].to_dict()
#             pert_results.append({
#                 'Identifier': identifier,
#                 'Sample': sample,
#                 'Age': sample_age,
#                 'Gene': gene,
#                 'Subtype': subtype,
#                 **diff_results})

#         # Revert to previous state
#         manager.load_state('steady')
#         manager.clear_perturbations()

# # Convert to DataFrame and save
# sort_order = ['Age', 'Sample', 'Gene', 'Subtype']
# pert_results = pd.DataFrame(pert_results).sort_values(sort_order)
# pert_results.to_csv('../plots/expval/perm_knockdown.csv')

In [10]:
# Load results
pert_results = pd.read_csv('../plots/expval/perm_knockdown.csv', index_col=0)

In [11]:
# # Compute gene effect sizes
# ges = []
# df = pert_results.copy()  # Just overwrite to save writes
# df['Total'] = df['Gene'].apply(len)
# for gene in tqdm.tqdm(adata.var_names):
#     # Precompute
#     df['Present'] = df['Gene'].apply(lambda l: gene in l)
#     df['Raw Weight'] = df['Present'] / df['Total']

#     def compute_scores(df):
#         # Compute scores
#         df['Weight'] = df['Raw Weight'] / df['Raw Weight'].sum()
#         gene_score = (df['Weight'] * df['Steady GES']).sum()
#         reliability = df['Raw Weight'].sum()
#         # df['Effect Size'] = gene_score
#         # df['Reliability'] = reliability
#         return pd.DataFrame(pd.DataFrame([{'Gene': gene, 'Effect Size': gene_score, 'Reliability': reliability}]))

#     ges.append(
#         df.groupby(['Subtype', 'Sample', 'Age'])
#         .apply(compute_scores, include_groups=False)
#         .reset_index().drop(columns='level_3'))

#     # Limit to subtype
#     # for subtype, (sample, age) in itertools.product(
#     #     pert_results['Subtype'].unique(),
#     #     pert_results.apply(lambda r: (r['Sample'], r['Age']), axis=1).unique(),
#     # ):
#     #     df['Subtype Weight'] = df['Raw Weight']
#     #     # Filter
#     #     inverse_mask = df['Subtype']!=subtype
#     #     inverse_mask += df['Sample']!=sample
#     #     inverse_mask += df['Age']!=age
#     #     if np.sum(~inverse_mask) == 0: continue  # Skip if none
#     #     df.loc[inverse_mask, 'Subtype Weight'] = 0

#     #     # Compute scores
#     #     df['Weight'] = df['Subtype Weight'] / df['Subtype Weight'].sum()
#     #     gene_score = (df['Weight'] * df['Steady GES']).sum()
#     #     reliability = df['Subtype Weight'].sum()

#     #     # Record
#     #     ges.append({
#     #         'Gene': gene,
#     #         'Subtype': subtype,
#     #         'Identifier': f'{age} ({sample})',
#     #         'Sample': sample,
#     #         'Age': age,
#     #         'Effect Size': gene_score,
#     #         'Reliability': reliability,
#     #     })

# # Postprocess and save
# # ges = pd.DataFrame(ges)
# ges = pd.concat(ges, axis=0)
# ges.to_csv('../plots/expval/ges.csv')


In [12]:
# Load results
ges = pd.read_csv('../plots/expval/ges.csv', index_col=0)

In [13]:
# Compute percentiles
def compute_percentile(df):
    df = df.sort_values('Effect Size', ascending=False)
    df['Rank'] = np.arange(len(df))+1
    df['Quantile'] = 1 - (df['Rank']-1) / (len(df)-1)
    df['Percentile'] = 100 * df['Quantile']
    return df
ges = ges.groupby('Subtype').apply(compute_percentile, include_groups=False)
ges = ges.reset_index().drop(columns='level_1')

In [14]:
df = ges.set_index('Gene').loc[['RAPGEF4', 'CHD8', 'NFIA', 'NFIB', 'MEIS2']]
df.loc[df['Percentile'] > 90]
# ges.loc[ges['Percentile'] > 80].groupby('Gene').size().sort_values(ascending=False)

,Subtype,Sample,Age,Effect Size,Reliability,Rank,Quantile,Percentile
Gene,,,,,,,,
RAPGEF4,All,10,155,816.316294,0.006008,11155,0.928769,92.876894
RAPGEF4,Astro EGFR,10,155,882.027208,0.006008,11008,0.929708,92.970771
RAPGEF4,Astro GFAP,10,155,924.279558,0.006008,11535,0.926342,92.634221
RAPGEF4,Astro MFGE8,10,155,833.071714,0.006008,11209,0.928424,92.842409
RAPGEF4,ExN L2-3,10,155,337.157972,0.006008,1954,0.987528,98.752786
...,...,...,...,...,...,...,...,...
MEIS2,GPC oIPC,8,145,597.851997,0.005949,14955,0.904502,90.450159
MEIS2,InN CGE,10,155,1337.948391,0.005949,15243,0.902662,90.266238
MEIS2,InN MGE,4,105,1114.559473,0.005949,15189,0.903007,90.300724


In [18]:
df.loc[df['Percentile'] > 90].loc['RAPGEF4']

,Subtype,Sample,Age,Effect Size,Reliability,Rank,Quantile,Percentile
Gene,,,,,,,,
RAPGEF4,All,10,155,816.316294,0.006008,11155,0.928769,92.876894
RAPGEF4,Astro EGFR,10,155,882.027208,0.006008,11008,0.929708,92.970771
RAPGEF4,Astro GFAP,10,155,924.279558,0.006008,11535,0.926342,92.634221
RAPGEF4,Astro MFGE8,10,155,833.071714,0.006008,11209,0.928424,92.842409
RAPGEF4,ExN L2-3,10,155,337.157972,0.006008,1954,0.987528,98.752786
RAPGEF4,ExN L2-3,3,100,335.231648,0.006008,14982,0.904329,90.432917
RAPGEF4,ExN L2-3/3-5,3,100,319.008370,0.006008,1713,0.978134,97.813370
RAPGEF4,ExN L3-5,3,100,372.684251,0.006008,10991,0.929816,92.981627
RAPGEF4,ExN L5,3,100,279.928195,0.006008,10333,0.905740,90.574025


In [19]:
df.loc[df['Percentile'] > 90].loc['CHD8']

,Subtype,Sample,Age,Effect Size,Reliability,Rank,Quantile,Percentile
Gene,,,,,,,,
CHD8,All,10,155,816.464215,0.007656,3171,0.979756,97.975592
CHD8,Astro EGFR,10,155,882.424422,0.007656,2853,0.981787,98.178672
CHD8,Astro GFAP,10,155,924.604453,0.007656,3262,0.979175,97.917478
CHD8,Astro MFGE8,10,155,833.424850,0.007656,3039,0.980599,98.059889
CHD8,Endo Capillary,7,125,4361.047069,0.007656,5763,0.954004,95.400372
CHD8,ExN L2-3,10,155,337.149396,0.007656,3290,0.978996,97.899597
CHD8,ExN L2-3/3-5,3,100,319.004142,0.007656,2235,0.971467,97.146652
CHD8,ExN L3-5,3,100,372.727141,0.007656,2851,0.981799,98.179949
CHD8,ExN L5,3,100,279.944577,0.007656,2902,0.973534,97.353392


In [20]:
df.loc[df['Percentile'] > 90].loc['MEIS2']

,Subtype,Sample,Age,Effect Size,Reliability,Rank,Quantile,Percentile
Gene,,,,,,,,
MEIS2,All,10,155,777.127140,0.005949,15005,0.904182,90.418229
MEIS2,Astro EGFR,10,155,816.907641,0.005949,14970,0.904406,90.440580
MEIS2,Astro GFAP,10,155,859.609605,0.005949,14973,0.904387,90.438664
MEIS2,Astro MFGE8,10,155,771.202404,0.005949,14971,0.904399,90.439942
MEIS2,ExN L2-3,10,155,335.571836,0.005949,13841,0.911616,91.161576
MEIS2,ExN L3-5,3,100,373.579323,0.005949,566,0.996392,99.639183
MEIS2,ExN L3-5,2,95,372.217180,0.005949,13449,0.914119,91.411913
MEIS2,ExN L6,9,155,402.556396,0.005949,218,0.998614,99.861421
MEIS2,GPC oIPC,8,145,597.851997,0.005949,14955,0.904502,90.450159


In [30]:
ges.loc[ges['Subtype']=='ExN L2-3']

,Subtype,Sample,Age,Gene,Effect Size,Reliability,Rank,Quantile,Percentile
751632,ExN L2-3,10,155,C16H17orf99,345.946746,0.006021,1,1.000000,100.000000
751633,ExN L2-3,10,155,SLC35A3,345.853952,0.006085,2,0.999994,99.999361
751634,ExN L2-3,10,155,GNG11,345.796630,0.006126,3,0.999987,99.998723
751635,ExN L2-3,10,155,PPM1L,345.645617,0.006235,4,0.999981,99.998084
751636,ExN L2-3,10,155,HSD11B2,345.625839,0.006249,5,0.999974,99.997446
...,...,...,...,...,...,...,...,...,...
908217,ExN L2-3,8,145,ASB3,221.150855,0.006102,156586,0.000026,0.002554
908218,ExN L2-3,6,125,STK10,220.945743,0.005410,156587,0.000019,0.001916
908219,ExN L2-3,6,125,MRPL37,220.450118,0.006085,156588,0.000013,0.001277
908220,ExN L2-3,6,125,ASB3,220.423335,0.006102,156589,0.000006,0.000639


In [21]:
# Load validation data
smart, = celltrip.utility.processing.read_adatas('s3://nkalafut-celltrip/ExpVal/smart_knockdown.h5ad', backed=True)

In [28]:
# Extract
smart_X = smart.X[:]
# Count normalize
smart_X = 10_000 * smart_X / smart_X.sum(axis=1)
# Log
smart_X = np.log1p(smart_X)
# Normalize
# smart_X_std = np.array(np.sqrt(smart_X.power(2).mean(axis=0) - np.square(smart_X.mean(axis=0))).reshape((1, -1)))
# smart_X_std = np.where(smart_X_std!=0, smart_X_std, 1)
# smart_X = (smart_X - smart_X.mean(axis=0)) / smart_X_std

In [27]:
smart.obs['acute_or_cultured'].unique()

['cultured_wo_virus_infection', 'shCHD8', 'shNC']
Categories (3, object): ['cultured_wo_virus_infection', 'shCHD8', 'shNC']

## Visualize

In [ ]:
# # Params
# subtypes_to_use = [
#     'all', 'Astro EGFR', 'Endo Capillary', 'ExN L2-3 IT CUX2 BRINP3',
#     'ExN L3-5 IT RORB ILRAPL2', 'ExN L5 ET POU3F1 RGMA', 'ExN L6 B NR4A2 ARHGAP6',
#     'InN CGE ADARB2 CCK', 'InN MGE SOX6 BMPER', 'InN dLGE MEIS2 KCNK13',
#     'MG', 'OPC', 'OPC PS', 'Oligo', 'VLMC']

# # Get labels and data
# pert_filter = pert_results
# # pert_filter = pert_filter.loc[pert_results['Subtype'].isin(subtypes_to_use)]
# pert_filter = pert_filter.sort_values(sort_order)
# xlabels, x = np.unique(pert_filter['Identifier'], return_inverse=True)
# x, xlabels = pd.factorize(pert_filter['Identifier'])  # Factorize keeps order, as opposed to `np.unique(..., return_inverse=True)`
# xticks = list(range(len(xlabels)))
# y, ylabels = pd.factorize(pert_filter['Gene'])
# yticks = list(range(len(ylabels)))
# z, zlabels = pd.factorize(pert_filter['Subtype'])
# zticks = list(range(len(zlabels)))
# values = pert_filter['Control GES'].to_numpy()
# values = values/values.max()

# # Create figure
# fig = plt.figure(figsize=(8, 16))
# ax = fig.add_subplot(projection='3d')

# # Plot heatmap
# ax.scatter(
#     x, y, z, c=[(1-v)*np.array((160/256, 200/256, 220/256, .4))+v*np.array((220/256, 150/256, 100/256, 1.)) for v in values], lw=0,
#     depthshade=False,  # Incompatible with alpha
#     s=200*values)

# # Orienting lines
# # for _x, _y in itertools.product(xticks, yticks):
# #     ax.plot([_x, _x], [_y, _y], [0, len(zticks)-1], color='k', alpha=.3, lw=1)
# # for _x, _z in itertools.product(xticks, zticks):
# #     ax.plot([_x, _x], [0, len(yticks)-1], [_z, _z], color='k', alpha=.3, lw=1)
# # for _y, _z in itertools.product(yticks, zticks):
# #     ax.plot([0, len(xticks)-1], [_y, _y], [_z, _z], color='k', alpha=.3, lw=1)
# for i in np.argwhere(values > .7).flatten():
#     ax.plot([x[i], 0], [y[i], y[i]], [z[i], z[i]], color='k', alpha=.6, lw=1, zorder=0)
#     ax.plot([x[i], x[i]], [y[i], len(yticks)-1], [z[i], z[i]], color='k', alpha=.6, lw=1, zorder=0)
#     ax.plot([x[i], x[i]], [y[i], y[i]], [z[i], 0], color='k', alpha=.6, lw=1, zorder=0)

# # Labels
# ax.set( 
#     xticks=xticks, xticklabels=xlabels,
#     yticks=yticks, yticklabels=ylabels,
#     zticks=zticks, zticklabels=zlabels)
# ax.set(xlim=[0, len(xticks)-1], ylim=[0, len(yticks)-1], zlim=[0, len(zticks)-1])

# # Save
# # ax.axis('off')
# fig.savefig('../plots/expval/heatmap.pdf')
# fig.show()

In [ ]:
# # Modified from https://python-graph-gallery.com/ridgeline-graph-seaborn/
# # Create grid
# pert_filter = pert_results
# # pert_filter = pert_filter.loc[pert_results['Subtype'].isin(subtypes_to_use)]  # Filter to select subtypes
# # pert_filter = pert_filter.loc[pert_filter['Gene']=='RAPGEF4']  # Filter to one gene
# pert_filter = pert_filter.copy().sort_values(['Subtype', 'Age', 'Sample'])  # Not the best sort
# subtype_gene_max = pert_filter.groupby(['Subtype', 'Gene'])['Control GES'].max()
# subtype_mean = pert_filter.groupby('Subtype')['Control GES'].mean()
# pert_filter['Subtype Gene Max Control GES'] = pert_filter[['Subtype', 'Gene']].apply(lambda x: subtype_gene_max[tuple(x)], axis=1).to_numpy()
# g = sns.FacetGrid(
#     pert_filter, row='Subtype', col='Gene', hue='Subtype Gene Max Control GES',
#     palette='coolwarm', aspect=15, height=0.75)
# # g.fig.patch.set_facecolor('None')
# for ax in g.axes.flat: ax.patch.set_facecolor('None')

# # Plot
# g.map(sns.lineplot, 'Identifier', 'Control GES', errorbar=None, color='white')
# def fillplot(*args, ax=None, alpha=1., **kwargs):
#     # Adapted from https://stackoverflow.com/a/77068597
#     if ax is None: ax = plt.gca()
#     for line in ax.lines:
#         x, y = line.get_xydata().T
#         ax.fill_between(x, 0, y, alpha=alpha, **kwargs)
# g.map(fillplot, 'Identifier', 'Control GES')

# # Labels and titles
# for (subtype, gene), ax in g.axes_dict.items():
#     is_first_gene = gene == pert_filter['Gene'].iloc[0]
#     is_first_subtype = subtype == pert_filter['Subtype'].iloc[0]
#     if is_first_gene:
#         ax.text(
#             -0.002, .002, subtype,
#             fontweight='bold', fontsize='large',
#             ha='right', va='bottom',
#             color='black',
#             transform=ax.transAxes)
#     if is_first_subtype:
#         ax.text(
#             .5, .5, gene,
#             fontweight='bold', fontsize='x-large',
#             ha='center', va='bottom',
#             color='black',
#             transform=ax.transAxes)
    
# # Remove extraneous formatting
# g.set_titles('')
# g.set(yticks=[], ylabel=None)
# g.despine(bottom=True, left=True)
# for ax in g.axes[:-1].flat: ax.tick_params(length=0)

# # Adjust
# g.set(ylim=(0, 1000))
# g.fig.subplots_adjust(hspace=-.8)
# g.fig.subplots_adjust(wspace=-.05)

# # Save
# g.fig.savefig('../plots/expval/effect_sizes.pdf')
# g.fig.show()

In [ ]:
# # Parameters
# traj_pairs = [
#     (3, 'RAPGEF4'),  #
#     (3, 'MEIS2'),
#     (3, 'NFIA'),
#     (3, 'NFIB'),
#     (3, 'CHD8'),
#     (3, 'MEIS2'),
#     (7, 'RAPGEF4'),
#     (7, 'MEIS2'),  #
#     (7, 'NFIA'),
#     (7, 'NFIB'),
#     (7, 'CHD8'),
#     (7, 'MEIS2'),
#     (10, 'RAPGEF4'),
#     (10, 'MEIS2'),
#     (10, 'NFIA'),  #
#     (10, 'NFIB'),
#     (10, 'CHD8'),
#     (10, 'MEIS2'),
# ]

# # Perturb
# trajectories = {}
# for traj_pair in traj_pairs:
#     # Split
#     sample, gene = traj_pair

#     # Filter adata
#     print(f'Testing {gene} on sample {sample}...')
#     adata_sample = adata[adata.obs['sample']==sample]
#     manager.set_modalities([adata_sample], suppress_warning=True)

#     # Simulate to steady state
#     manager.reset_env()
#     manager.simulate(skip_time=512., impute=False)
#     manager.save_state('steady')

#     # Perturb chosen gene
#     manager.add_perturbation([gene], modality=0, feature_targets=0)
#     time, states = manager.simulate_perturbation(skip_time=2., time=10., progress_bar=True)
#     trajectories[traj_pair] = (time, states[0])

In [ ]:
# # Lines
# for traj_pair in traj_pairs:
#     # Split
#     sample, gene = traj_pair
#     sample_obs = adata[adata.obs['sample']==sample].obs
#     sample_age = sample_obs['age'].iloc[0]

#     # Create figure
#     fig, ax = plt.subplots(1, 1, figsize=(6, 6))

#     # Get data
#     time, gex = trajectories[traj_pair]
#     gex_embed = red.transform(gex.reshape((-1, gex.shape[-1]))).reshape((*gex.shape[:-1], 2))

#     # Plot
#     subtypes = adata.obs['subtype'].unique()
#     palette = sns.color_palette(palette='husl', n_colors=len(subtypes))
#     for cell_i in range(gex_embed.shape[1]):
#         subtype = sample_obs['subtype'].iloc[cell_i]
#         ax.scatter(
#             *gex_embed[0, cell_i],
#             color=palette[(subtypes==subtype).argmax()],
#             label=subtype, s=2)
#         ax.plot(
#             *gex_embed[:, cell_i].T,
#             color=palette[(subtypes==subtype).argmax()],
#             alpha=.5, lw=.2)
#     ax.set_title(f'{gene} - {sample_age} ({sample})')
#     sns.despine(ax=ax)

#     # Save
#     fig.savefig(f'../plots/expval/{sample}_{gene}.pdf')


In [ ]:
# # Plot ct legend
# fig, ax = plt.subplots(1, 1)
# handles = [
#     plt.scatter(None, None, color=palette[(subtypes==subtype).argmax()], label=subtype)
#     for subtype in subtypes
# ]
# ax.legend(handles=handles)
# ax.axis('off');
# fig.savefig('../plots/expval/ct_legend.pdf')